<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Трансформация-категорийных-признаков" data-toc-modified-id="Трансформация-категорийных-признаков-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Трансформация категорийных признаков</a></span></li><li><span><a href="#Трансформация-количественных-признаков" data-toc-modified-id="Трансформация-количественных-признаков-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Трансформация количественных признаков</a></span></li><li><span><a href="#Обединение-признаков-в-вектор" data-toc-modified-id="Обединение-признаков-в-вектор-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Обединение признаков в вектор</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Обучение-модели-на-полной-выборке" data-toc-modified-id="Обучение-модели-на-полной-выборке-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение модели на полной выборке</a></span></li><li><span><a href="#Обучение-модели-на-числовых-пременных" data-toc-modified-id="Обучение-модели-на-числовых-пременных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение модели на числовых пременных</a></span></li></ul></li><li><span><a href="#Анализ-результатов" data-toc-modified-id="Анализ-результатов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ результатов</a></span><ul class="toc-item"><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Выводы</a></span></li></ul></li></ul></div>

# Предсказание стоимости жилья

**Задача** 

Обучить модель линейной регрессии. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Для оценки качества модели используйте метрики RMSE, MAE и R2.

**Данные**

Данные о жилье в Калифорнии в 1990 году.

## Подготовка данных

In [1]:
# # Загружаем библиотеки.
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator

In [2]:
RANDOM_SEED = 12345

In [2]:
pd.read_csv('/datasets/housing.csv').to_csv('housing.csv', index=False)

In [3]:
# Создаем локальную Spark-сессию.
spark = SparkSession.builder \
                    .master("local") \
                    .appName("California - Linear Regression") \
                    .getOrCreate()

In [4]:
df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) # Загружаем данные.

In [5]:
df.printSchema() # Выводим типы данных.

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



### Обработка пропусков

In [6]:
columns = df.columns # Считаем пропуски.

for column in columns:
    print(column, df.filter(F.col(column).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


In [7]:
df.select('ocean_proximity').distinct().collect()

[Row(ocean_proximity='ISLAND'),
 Row(ocean_proximity='NEAR OCEAN'),
 Row(ocean_proximity='NEAR BAY'),
 Row(ocean_proximity='<1H OCEAN'),
 Row(ocean_proximity='INLAND')]

In [8]:
df = df.na.fill(0) # Заполняем пропуски 0. 

In [9]:
print(df.filter(F.col('total_bedrooms').isNull()).count()) # Проверяем заполнение. 

0


### Трансформация категорийных признаков

In [10]:
# Списки с переменными.

categorical_cols = ['ocean_proximity']
numerical_cols  = [
    'longitude',
    'latitude',
    'housing_median_age',
    'total_rooms', 
    'total_bedrooms',
    'population',
    'households',
    'median_income'
]
target = 'median_house_value'

In [11]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

16431 4209


In [12]:
# Трансформируем категориальные переменные в числовые значения. 
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols]) 
fit_indexer = indexer.fit(train_data)
train_data = fit_indexer.transform(train_data)
test_data = fit_indexer.transform(test_data)

In [13]:
# Проводим OHE-кодирование. 
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])

fit_encoder = encoder.fit(train_data)

train_data = fit_encoder.transform(train_data)
test_data = fit_encoder.transform(test_data)

### Трансформация количественных признаков

In [14]:
# Объединяем количественные переменные.
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")

train_data = numerical_assembler.transform(train_data) 
test_data = numerical_assembler.transform(test_data) 

In [15]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol="numerical_features_scaled")

fit_standard = standardScaler.fit(train_data)

train_data = fit_standard.transform(train_data)
test_data = fit_standard.transform(test_data)

### Обединение признаков в вектор

In [16]:
# Объединяем категориальные переменные.
categorical_assembler = VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols], outputCol="categorical_features")
train_data = categorical_assembler.transform(train_data) 

In [17]:
# Объединяем категориальные переменные.
test_data = categorical_assembler.transform(test_data) 

In [18]:
# Объединяем все переменные.
all_features = ['categorical_features','numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 
train_data = final_assembler.transform(train_data)
test_data = final_assembler.transform(test_data)

train_data.select(all_features).show(3)
test_data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-62.004132451563...|
|       (4,[2],[1.0])|     [-61.979201155845...|
|       (4,[2],[1.0])|     [-61.979201155845...|
+--------------------+-------------------------+
only showing top 3 rows

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[2],[1.0])|     [-61.944297341839...|
|       (4,[2],[1.0])|     [-61.914379786977...|
|       (4,[2],[1.0])|     [-61.914379786977...|
+--------------------+-------------------------+
only showing top 3 rows



## Обучение моделей

### Обучение модели на полной выборке

In [19]:
lr = LinearRegression(labelCol=target, featuresCol='features', regParam=0.3)

model_all = lr.fit(train_data)

22/09/19 08:55:14 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/09/19 08:55:14 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/09/19 08:55:14 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/09/19 08:55:14 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


### Обучение модели на числовых пременных

In [20]:
lr = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled', regParam=0.3)

model_number = lr.fit(train_data)

## Анализ результатов

In [21]:
predictions_all = model_all.transform(test_data) # Делаем предсказание на модели по всем параметрам.

In [22]:
#  Считаем метрики для модели по всем параметрам.
rmse_all = RegressionEvaluator(labelCol='median_house_value', metricName='rmse').evaluate(predictions_all)
r2_all = RegressionEvaluator(labelCol='median_house_value', metricName='r2').evaluate(predictions_all)
mae_all = RegressionEvaluator(labelCol='median_house_value', metricName='mae').evaluate(predictions_all)

In [23]:
predictions_number = model_number.transform(test_data) # Делаем предсказание на модели на количественных параметрах.

In [24]:
# Делаем предсказание на модели на количественных параметрах.
rmse_number = RegressionEvaluator(labelCol='median_house_value', metricName='rmse').evaluate(predictions_number)
r2_number = RegressionEvaluator(labelCol='median_house_value', metricName='r2').evaluate(predictions_number)
mae_number = RegressionEvaluator(labelCol='median_house_value', metricName='mae').evaluate(predictions_number)

In [25]:
columns = ['model', 'RMSE','R2','MAE']

data = [
    ['ALL',  rmse_all, r2_all, mae_all],
    ['Number', rmse_number, r2_number, mae_number]]

pd.DataFrame(data=data, columns=columns)

,model,RMSE,R2,MAE
0,ALL,67738.163611,0.658559,49173.354949
1,Number,68866.435291,0.647090,50418.942807


### Выводы

- Показатели обучения модели только на числовых данных оказались хуже, хоть и не значительно.
- Параметр удалённости от океана влияет на стоимость жилья и игнорировать этот параметр нельзя.
- Частично он компенсируется наличием широты и долготы, но впрямую использовать эти параметры то же не очень практично.